In [ ]:
!pip install barbar

import numpy as np
import pandas as pd
import time
import csv
import random
import matplotlib.pyplot as plt
from PIL import Image
from barbar import Bar

import torch
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split

import sklearn.metrics as metrics
from sklearn.metrics import roc_auc_score

use_gpu = torch.cuda.is_available()
print(use_gpu)

True


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!unzip /content/gdrive/MyDrive/ai_for_healthcare/archive.zip

Streaming output truncated to the last 5000 lines.
  inflating: CheXpert-v1.0-small/train/patient60445/study2/view1_frontal.jpg  
  inflating: CheXpert-v1.0-small/train/patient60445/study2/view2_lateral.jpg  
  inflating: CheXpert-v1.0-small/train/patient60446/study1/view1_frontal.jpg  
  inflating: CheXpert-v1.0-small/train/patient60446/study2/view1_frontal.jpg  
  inflating: CheXpert-v1.0-small/train/patient60446/study2/view2_lateral.jpg  
  inflating: CheXpert-v1.0-small/train/patient60447/study1/view1_frontal.jpg  
  inflating: CheXpert-v1.0-small/train/patient60447/study1/view2_lateral.jpg  
  inflating: CheXpert-v1.0-small/train/patient60448/study1/view1_frontal.jpg  
  inflating: CheXpert-v1.0-small/train/patient60448/study1/view2_lateral.jpg  
  inflating: CheXpert-v1.0-small/train/patient60448/study1/view3_lateral.jpg  
  inflating: CheXpert-v1.0-small/train/patient60449/study1/view1_frontal.jpg  
  inflating: CheXpert-v1.0-small/train/patient60449/study1/view2_lateral.jpg  
 

**Pre-Defined Values**

In [ ]:
columns_to_drop = ['Sex', 'Age', 'Frontal/Lateral', 'AP/PA', 'No Finding', 'Enlarged Cardiomediastinum', 'Lung Opacity', 'Lung Lesion', 'Consolidation', 'Atelectasis', 'Pneumothorax', 'Pleural Effusion', 'Pleural Other', 'Fracture', 'Support Devices']

Traindata = pd.read_csv('CheXpert-v1.0-small/train.csv') # read train.csv
Traindata = Traindata.drop(columns=columns_to_drop) # Getting rid of everything except the 3 diseases that we need
Traindata = Traindata[Traindata['Path'].str.contains("frontal")] # use only frontal images
Traindata = Traindata[500:] # data 500 to the end
Traindata.to_csv('./CheXpert-v1.0-small/train_mod.csv', index = False) #Convert to CSV
print("Train data length:", len(Traindata))
print(Traindata)

Validdata = pd.read_csv('./CheXpert-v1.0-small/valid.csv') # read valid.csv
Validdata = Validdata.drop(columns=columns_to_drop) # Getting rid of everything except the 3 diseases that we need
Validdata = Validdata[Validdata['Path'].str.contains("frontal")] # use only frontal images
Validdata.to_csv('./CheXpert-v1.0-small/valid_mod.csv', index = False) #Convert to CSV
print("Valid data length:", len(Validdata))

Testdata = Traindata[:500] # train data is 500 to the end
Testdata.to_csv('./CheXpert-v1.0-small/test_mod.csv', index = False) #Convert to CSV
print("Test data length:", len(Testdata))

pathFileTrain = './CheXpert-v1.0-small/train_mod.csv'
pathFileValid = './CheXpert-v1.0-small/valid_mod.csv'
pathFileTest = './CheXpert-v1.0-small/test_mod.csv'

nnIsTrained = False     # Not pre-trained
nnClassCount = 4        # Dimension of the output (edema, cardiomegaly, pneumonia, nothing)

trBatchSize = 16        # Batch size
trMaxEpoch = 3          # Epoch

# Image transformation here
imgtransResize = (320, 320)
imgtransCrop = 224

class_names = ['Cardiomegaly', 'Edema', 'Pneumonia', 'Nothing']

Train data length: 190527
                                                     Path  Cardiomegaly  \
621     CheXpert-v1.0-small/train/patient00147/study13...           NaN   
622     CheXpert-v1.0-small/train/patient00147/study8/...           NaN   
623     CheXpert-v1.0-small/train/patient00147/study14...           NaN   
624     CheXpert-v1.0-small/train/patient00147/study10...           NaN   
625     CheXpert-v1.0-small/train/patient00147/study9/...           NaN   
...                                                   ...           ...   
223409  CheXpert-v1.0-small/train/patient64537/study2/...           NaN   
223410  CheXpert-v1.0-small/train/patient64537/study1/...           NaN   
223411  CheXpert-v1.0-small/train/patient64538/study1/...           NaN   
223412  CheXpert-v1.0-small/train/patient64539/study1/...           1.0   
223413  CheXpert-v1.0-small/train/patient64540/study1/...           NaN   

        Edema  Pneumonia  
621       1.0        NaN  
622       1.0      

**Create Dataset**

In [ ]:
class CheXpertDataSet(Dataset):
    def __init__(self, data_PATH, transform = None, policy = "ones"):
      image_names = []
      labels = []
      with open(data_PATH, "r") as f:
            csvReader = csv.reader(f)
            next(csvReader, None) # skip the header
            for line in csvReader:
              image_name = line[0]
              label = line[1:] + [0]

              for i in range(4):
                if i == 3:
                  if label[0] == 1 or label[1] == 1 or label[2] == 1:
                    label[i] = 0
                  else:
                    label[i] = 1
                else:
                  if label[i]:
                    a = float(label[i])
                    if a == 1:
                      label[i] = 1
                    elif a == -1:
                      if policy == "ones":
                        label[i] = 1
                      elif policy == "zeroes":
                        label[i] = 0
                      else:
                        label[i] = 0
                    else:
                      label[i] = 0
                  else:
                    label[i] = 0

              image_names.append('./' + image_name)
              labels.append(label)

      self.image_names = image_names
      self.labels = labels
      self.transform = transform

    def __getitem__(self, index):
      """Take the index of item and returns the image and its labels"""
      image_name = self.image_names[index]
      image = Image.open(image_name).convert('RGB')
      label = self.labels[index]
      if self.transform is not None:
        image = self.transform(image)
      return image, torch.FloatTensor(label)

    def __len__(self):
      return len(self.image_names)

**Load Datasets**

In [ ]:
# BELOW ARE SOME TRANSFORMATION CODES

IMAGENET_MEAN = [0.485, 0.456, 0.406]  # mean of ImageNet dataset(for normalization)
IMAGENET_STD = [0.229, 0.224, 0.225]   # std of ImageNet dataset(for normalization) DO NOT KNOW WHAT THIS MEANS

# Tranform data
normalize = transforms.Normalize(IMAGENET_MEAN, IMAGENET_STD)
transformList = []

transformList.append(transforms.Resize((imgtransCrop, imgtransCrop))) # 224
# transformList.append(transforms.RandomResizedCrop(imgtransCrop))
# transformList.append(transforms.RandomHorizontalFlip())
transformList.append(transforms.ToTensor())
# transformList.append(normalize)
transformSequence = transforms.Compose(transformList)

print(transformList)

datasetTrain = CheXpertDataSet(pathFileTrain, transformSequence, policy = "ones")
print("Train data length:", len(datasetTrain))
print(datasetTrain[0])

datasetValid = CheXpertDataSet(pathFileValid, transformSequence)
print("Valid data length:", len(datasetValid))

datasetTest = CheXpertDataSet(pathFileTest, transformSequence, policy = "ones")
print("Test data length:", len(datasetTest))

NameError: ignored

**Create Train and Test Models**

In [ ]:
class CheXpertTrainer():
  def train(model, dataLoaderTrain, dataLoaderVal, nnClassCount, trMaxEpoch, checkpoint):
    # optimizations (1: specifies parameters to be optimized, lr: learning rate, betas: coefficients used for running averages of gradients, weight_decay: penalty for large weights)
    optimizer = optim.Adam(model.parameters(), lr = 0.0001, betas = (0.9, 0.999), eps = 1e-08, weight_decay = 0)
    loss = torch.nn.BCELoss() # setting loss function

    if checkpoint != None and use_gpu: # loading checkpoint
            modelCheckpoint = torch.load(checkpoint)
            model.load_state_dict(modelCheckpoint['state_dict'])
            optimizer.load_state_dict(modelCheckpoint['optimizer'])

    # Train the network
    lossMIN = 100000
    train_start = []
    train_end = []
    for epochID in range(0, trMaxEpoch):
        train_start.append(time.time()) # training starts
        losst = CheXpertTrainer.epochTrain(model, dataLoaderTrain, optimizer, trMaxEpoch, nnClassCount, loss)
        train_end.append(time.time()) # training ends
        lossv = CheXpertTrainer.epochVal(model, dataLoaderVal, optimizer, trMaxEpoch, nnClassCount, loss)
        print("Training loss: {:.3f},".format(losst), "Valid loss: {:.3f}".format(lossv))

        if lossv < lossMIN:
            lossMIN = lossv
            torch.save({'epoch': epochID + 1, 'state_dict': model.state_dict(),
                        'best_loss': lossMIN, 'optimizer' : optimizer.state_dict()},
                        'm-epoch_FL' + str(epochID + 1) + '.pth.tar')
            print('Epoch ' + str(epochID + 1) + ' [save] loss = ' + str(lossv))
        else:
            print('Epoch ' + str(epochID + 1) + ' [----] loss = ' + str(lossv))

    train_time = np.array(train_end) - np.array(train_start)
    print("Training time for each epoch: {} seconds".format(train_time.round(0)))
    params = model.state_dict()
    return params

  def epochTrain(model, dataLoaderTrain, optimizer, epochMax, classCount, loss):
    losstrain = 0
    model.train()

    for batchID, (varInput, target) in enumerate(Bar(dataLoaderTrain)):

        varTarget = target.cuda(non_blocking = True)
        varOutput = model(varInput)
        lossvalue = loss(varOutput, varTarget)

        optimizer.zero_grad()
        lossvalue.backward()
        optimizer.step()

        losstrain += lossvalue.item()

    return losstrain / len(dataLoaderTrain)

  def epochTrain(model, dataLoaderTrain, optimizer, epochMax, classCount, loss):
    losstrain = 0
    model.train()

    for batchID, (varInput, target) in enumerate(Bar(dataLoaderTrain)):

        varTarget = target.cuda(non_blocking = True)
        varOutput = model(varInput)
        lossvalue = loss(varOutput, varTarget)

        optimizer.zero_grad()
        lossvalue.backward()
        optimizer.step()

        losstrain += lossvalue.item()

    return losstrain / len(dataLoaderTrain)

  def epochVal(model, dataLoaderVal, optimizer, epochMax, classCount, loss):
        model.eval()
        lossVal = 0

        with torch.no_grad():
            for i, (varInput, target) in enumerate(dataLoaderVal):

                target = target.cuda(non_blocking = True)
                varOutput = model(varInput)

                lossVal += loss(varOutput, target)

        return lossVal / len(dataLoaderVal)


  def computeAUROC(dataGT, dataPRED, classCount):
    # Computes area under ROC curve
    # dataGT: ground truth data
    # dataPRED: predicted data
    outAUROC = []
    datanpGT = dataGT.cpu().numpy()
    datanpPRED = dataPRED.cpu().numpy()

    for i in range(classCount):
        try:
            outAUROC.append(roc_auc_score(datanpGT[:, i], datanpPRED[:, i]))
        except ValueError:
            pass
    return outAUROC

  def test(model, dataLoaderTest, nnClassCount, checkpoint, class_names):
      cudnn.benchmark = True

      if checkpoint != None and use_gpu:
          modelCheckpoint = torch.load(checkpoint)
          model.load_state_dict(modelCheckpoint['state_dict'])

      if use_gpu:
          outGT = torch.FloatTensor().cuda()
          outPRED = torch.FloatTensor().cuda()
      else:
          outGT = torch.FloatTensor()
          outPRED = torch.FloatTensor()

      model.eval()

      with torch.no_grad():
          for i, (input, target) in enumerate(dataLoaderTest):

              target = target.cuda()
              outGT = torch.cat((outGT, target), 0).cuda()

              bs, c, h, w = input.size()
              varInput = input.view(-1, c, h, w)

              out = model(varInput)
              outPRED = torch.cat((outPRED, out), 0)
      aurocIndividual = CheXpertTrainer.computeAUROC(outGT, outPRED, nnClassCount)
      aurocMean = np.array(aurocIndividual).mean()
      print('AUROC mean ', aurocMean)

      for i in range (0, len(aurocIndividual)):
          print(class_names[i], ' ', aurocIndividual[i])

      return outGT, outPRED

**Define the Model**

In [ ]:
class DenseNet121(nn.Module):
    """Model modified.
    The architecture of our model is the same as standard DenseNet121
    except the classifier layer which has an additional sigmoid function.
    """
    def __init__(self, out_size):
        super(DenseNet121, self).__init__()
        self.densenet121 = torchvision.models.densenet121(pretrained = False)
        num_ftrs = self.densenet121.classifier.in_features
        self.densenet121.classifier = nn.Sequential(
            nn.Linear(num_ftrs, out_size),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.densenet121(x)
        return x

[link text](https://)**Train the model with Federated Learning**



In [ ]:
# FOR MULTIPLE COMMUNICATION ROUNDS
com_round = 3
fraction = 1.0
epoch = 3
batch = 48
num_clients = 5

'''
# Divide datasetTrain_ex
datasetTrain_1, datasetTrain_2, datasetTrain_3, datasetTrain_4, datasetTrain_5, dataleft = random_split(datasetTrain,
                                                                                                        [100, 100, 100, 100, 100,
                                                                                                         len(datasetTrain) - 500])
'''
# Divide datasetTrain_real
datasetTrain_1, datasetTrain_2, datasetTrain_3, datasetTrain_4, datasetTrain_5 = random_split(datasetTrain,
                                                                                              [38106, 38106, 38105, 38105, 38105])


# Define 5 DataLoaders
dataLoaderTrain_1 = DataLoader(dataset = datasetTrain_1, batch_size = trBatchSize,
                               shuffle = True, num_workers = 2, pin_memory = True)
dataLoaderTrain_2 = DataLoader(dataset = datasetTrain_2, batch_size = trBatchSize,
                               shuffle = True, num_workers = 2, pin_memory = True)
dataLoaderTrain_3 = DataLoader(dataset = datasetTrain_3, batch_size = trBatchSize,
                               shuffle = True, num_workers = 2, pin_memory = True)
dataLoaderTrain_4 = DataLoader(dataset = datasetTrain_4, batch_size = trBatchSize,
                               shuffle = True, num_workers = 2, pin_memory = True)
dataLoaderTrain_5 = DataLoader(dataset = datasetTrain_5, batch_size = trBatchSize,
                               shuffle = True, num_workers = 2, pin_memory = True)

# Define Valid and Test DataLoaders
dataLoaderVal = DataLoader(dataset = datasetValid, batch_size = trBatchSize,
                           shuffle = False, num_workers = 2, pin_memory = True)
dataLoaderTest = DataLoader(dataset = datasetTest, num_workers = 2, pin_memory = True)

In [ ]:
dT = [datasetTrain_1, datasetTrain_2, datasetTrain_3, datasetTrain_4, datasetTrain_5]
dLT = [dataLoaderTrain_1, dataLoaderTrain_2, dataLoaderTrain_3, dataLoaderTrain_4, dataLoaderTrain_5]

model = DenseNet121(nnClassCount).cuda() # Step 0: Initialize global model and load the model
model = torch.nn.DataParallel(model).cuda()

for i in range(com_round):
    print("[[[ Round {} Start ]]]".format(i + 1))
    params = [None] * num_clients
    sel_clients = sorted(random.sample(range(num_clients),
                                       round(num_clients*fraction))) # Step 1: select random fraction of clients
    print("The number of clients:", len(sel_clients))
    for j in sel_clients: # Step 2: send weights to clients
        print("<< Client {} Training Start >>".format(j + 1))
        train_valid_start = time.time()
        params[j] = CheXpertTrainer.train(model, dLT[j], dataLoaderVal, # Step 3: Perform local computations
                                          nnClassCount, trMaxEpoch = epoch, checkpoint = None)
        train_valid_end = time.time()
        client_time = round(train_valid_end - train_valid_start)
        print("<< Client {} Training End: {} seconds elapsed >>".format(j + 1, client_time))

    fidx = [idx for idx in range(len(params)) if params[idx] != None][0]
    lidx = [idx for idx in range(len(params)) if params[idx] != None][-1]
    for key in params[fidx]: # Step 4: return updates to server
        weights, weightn = [], []
        for k in sel_clients:
            weights.append(params[k][key]*len(dT[k]))
            weightn.append(len(dT[k]))
        params[lidx][key] = sum(weights) / sum(weightn) # weighted averaging model weights

    model = DenseNet121(nnClassCount).cuda()
    model = torch.nn.DataParallel(model).cuda()
    model.load_state_dict(params[lidx]) # Step 5: server updates global state
    print("[[[ Round {} End ]]]".format(i + 1))

print("Global model trained")

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


[[[ Round 1 Start ]]]
The number of clients: 5
<< Client 1 Training Start >>
38106/38106: [===============================>] - ETA 3.6s
Training loss: 0.473, Valid loss: 0.460
Epoch 1 [save] loss = tensor(0.4600, device='cuda:0')
38106/38106: [===============================>] - ETA 0.6s
Training loss: 0.451, Valid loss: 0.454
Epoch 2 [save] loss = tensor(0.4545, device='cuda:0')
38106/38106: [===============================>] - ETA 0.4s
Training loss: 0.442, Valid loss: 0.432
Epoch 3 [save] loss = tensor(0.4318, device='cuda:0')
Training time for each epoch: [472. 478. 478.] seconds
<< Client 1 Training End: 1433 seconds elapsed >>
<< Client 2 Training Start >>
38106/38106: [===============================>] - ETA 0.4s
Training loss: 0.444, Valid loss: 0.437
Epoch 1 [save] loss = tensor(0.4368, device='cuda:0')
38106/38106: [===============================>] - ETA 0.6s
Training loss: 0.436, Valid loss: 0.438
Epoch 2 [----] loss = tensor(0.4376, device='cuda:0')
38106/38106: [=========